In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyproj import CRS, Transformer
from matplotlib.lines import Line2D
from math import radians, cos, sin, atan2
import os
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"
import h5py


In [ ]:
# Load the data
df = pd.read_csv('Paradise2NisquallyEntrace_Channels_interp_direction.csv')
dfslope = pd.read_csv('mora-das-slope.csv')

# Drop rows with NaN values in Longitude or Latitude columns
df = df.dropna(subset=['Longitude [°]', 'Latitude [°]']).reset_index(drop=True)

# Define the color mapping based on lithology
lithology_colors = {
    'Qs': '#FFD700',    # Gold
    'Qra': '#DC143C',   # Deep Pink
    'To': '#8B4513',     # Khaki
    'Tg': '#FF8C00',    # Dark Orange
    'Tdi': '#DC143C',   # Crimson
    'Td': '#9370DB',    # Medium Purple
}

# Define coordinate transformations
crs_wgs84 = CRS("EPSG:4326")     # WGS84 latitude and longitude
crs_utm10n = CRS("EPSG:32610")   # UTM zone 10N

# Create a transformer object
transformer = Transformer.from_crs(crs_wgs84, crs_utm10n, always_xy=True)

# Convert longitude and latitude to x and y in meters
x_meters, y_meters = transformer.transform(df['Longitude [°]'].values, df['Latitude [°]'].values)
df['x_meters'] = x_meters
df['y_meters'] = y_meters

# Convert meters to kilometers
df['x'] = df['x_meters'] / 1000
df['y'] = df['y_meters'] / 1000

# Earthquake coordinates (lat, lon) and convert to x, y in kilometers
quake_lat = 46.7722
quake_lon = -121.9588 
quake_x_meters, quake_y_meters = transformer.transform(quake_lon, quake_lat)
quake_x = quake_x_meters / 1000
quake_y = quake_y_meters / 1000

# Compute the midpoints between consecutive fiber points
lat_fibra = df['Latitude [°]'].values
lon_fibra = df['Longitude [°]'].values

lat_midpoints = (lat_fibra[1:] + lat_fibra[:-1]) / 2
lon_midpoints = (lon_fibra[1:] + lon_fibra[:-1]) / 2

# Convert midpoints to x, y in kilometers
x_mid_meters, y_mid_meters = transformer.transform(lon_midpoints, lat_midpoints)
x_mid = x_mid_meters / 1000
y_mid = y_mid_meters / 1000

# Compute azimuths and orientations
def compute_theta(x1, y1, x2, y2):
    return np.arctan2(y2 - y1, x2 - x1)

# Fiber orientation angles (theta_fiber)
theta_fiber = compute_theta(df['x'].values[:-1], df['y'].values[:-1], df['x'].values[1:], df['y'].values[1:])

# Direction from earthquake to midpoints (theta_quake)
theta_quake = compute_theta(np.full_like(x_mid, quake_x), np.full_like(y_mid, quake_y), x_mid, y_mid)

# Difference in angles
theta_diff = theta_quake - theta_fiber

# Ensure theta_diff is between -pi and pi
theta_diff = (theta_diff + np.pi) % (2 * np.pi) - np.pi

# Radial and tangential components
radial_component = np.cos(theta_diff) ** 2
tangential_component = np.sin(2 * theta_diff)

# Normalize components for plotting
radial_normalized = (radial_component - np.min(radial_component)) / (np.max(radial_component) - np.min(radial_component))
tangential_normalized = (tangential_component - np.min(tangential_component)) / (np.max(tangential_component) - np.min(tangential_component))


In [ ]:
# Parameters
chan_min = 0  # Starting channel index
chan_max = -1  # Use all channels
file = '/1-fnp/petasaur/p-jbod1/rainier/2023/08/27/decimator_2023-08-27_10.10.00_UTC.h5'

# Load data
with h5py.File(file, 'r') as rawdata:
    data = np.array(rawdata['Acquisition/Raw[0]/RawData'][:, chan_min:chan_max])
    #print(" Unidad de los datos:", data.attrs.get('Unit'))
    attrs= dict(rawdata['Acquisition'].attrs)

In [ ]:
dfslope
slope = dfslope['slope_deg']

In [ ]:
opdist=df['Optical Distance [m]']

In [ ]:
# Step 1: Load signal and noise data from .npy files
# =============================================================================
selected_data_p = np.load('pwave.npy')      # P-wave window (time x channels)
selected_data_s = np.load('swave.npy')      # S-wave window (time x channels)
selected_data_n = np.load('noisetest.npy')  # Noise window (time x channels)

# =============================================================================
# Step 2: Calculate RMS (Root Mean Square) for signal and noise
# =============================================================================
# RMS for P-wave, S-wave, and Noise
rms_pwave = np.sqrt(np.nanmean(selected_data_p**2, axis=0))
rms_swave = np.sqrt(np.nanmean(selected_data_s**2, axis=0))
rms_noise = np.sqrt(np.nanmean(selected_data_n**2, axis=0))

# =============================================================================
# Step 3: Add RMS results to the dataframe
# =============================================================================
df['AmplitudeP'] = rms_pwave[:len(df)]
df['AmplitudeS'] = rms_swave[:len(df)]
df['AmplitudeN'] = rms_noise[:len(df)]
print("RMS amplitudes correctly added to dataframe.")

# =============================================================================
# Step 4: Apply strain-rate correction
# =============================================================================
gl = 9.571428805203961  # Gauge Length in meters
correction_factor = (1550 * 1e-9) / (0.78 * 4 * np.pi * 1.4682 * gl)
print("Correction factor:", correction_factor)

# Apply correction to RMS amplitudes (convert to strain rate units)
df['AmplitudeP_corr'] = df['AmplitudeP'] * correction_factor
df['AmplitudeS_corr'] = df['AmplitudeS'] * correction_factor
df['AmplitudeN_corr'] = df['AmplitudeN'] * correction_factor

# =============================================================================
# Step 5: Calculate SNR (Signal-to-Noise Ratio) for each wave
# =============================================================================
# SNR is defined as RMS(signal) / RMS(noise)
df['SNR_P'] = df['AmplitudeP'] / df['AmplitudeN']
df['SNR_S'] = df['AmplitudeS'] / df['AmplitudeN']

# =============================================================================
# Step 6: Plot SNR as a function of optical distance
# =============================================================================
fig, axes = plt.subplots(2, 1, figsize=(18, 10))

# Subplot 1: SNR for P-wave
axes[0].scatter(df['Optical Distance [m]'] / 1000, df['SNR_P'], color='red', alpha=0.5)
axes[0].scatter(df['Optical Distance [m]'] / 1000, df['AmplitudeN'], color='gray', alpha=0.1, label='Noise floor')
axes[0].set_ylabel('SNR P-wave', fontsize=30)
axes[0].set_xlabel('Optical Distance (km)', fontsize=30)
axes[0].set_yscale('log')
axes[0].legend(loc='upper right', fontsize=20)
axes[0].grid(True, linestyle='--', linewidth=0.8, alpha=0.5)
plt.setp(axes[0].get_xticklabels(), fontsize=25)
plt.setp(axes[0].get_yticklabels(), fontsize=25)

# Subplot 2: SNR for S-wave
axes[1].scatter(df['Optical Distance [m]'] / 1000, df['SNR_S'], color='blue', alpha=0.5)
axes[1].scatter(df['Optical Distance [m]'] / 1000, df['AmplitudeN'], color='gray', alpha=0.1, label='Noise floor')
axes[1].set_ylabel('SNR S-wave', fontsize=30)
axes[1].set_xlabel('Optical Distance (km)', fontsize=30)
axes[1].set_yscale('log')
axes[1].legend(loc='upper right', fontsize=20)
axes[1].grid(True, linestyle='--', linewidth=0.8, alpha=0.5)
plt.setp(axes[1].get_xticklabels(), fontsize=25)
plt.setp(axes[1].get_yticklabels(), fontsize=25)

# Adjust layout and save figure
plt.tight_layout()
plt.savefig('snr_waves-log.png', dpi=300, bbox_inches='tight', format='png')
plt.show()


In [ ]:
import matplotlib.colors as mcolors
df_mid = pd.DataFrame({
    'Latitude [°]': lat_fibra[0:len(radial_component)],
    'Longitude [°]': lon_fibra[0:len(radial_component)],
    'x': x_mid,
    'y': y_mid,
    'Radial Component': radial_component,
    'Tangential Component': tangential_component,
    'Radial Normalized': radial_normalized,
    'Tangential Normalized': tangential_normalized,
    'Optical distance': opdist[0:len(radial_component)],
    'Amplitude-P' :rms_pwave[0:len(radial_component)],
    'Amplitude-S' :rms_swave[0:len(radial_component)],
    'Noise' : rms_noise[0:len(radial_component)],
    'Slope' : slope[0:len(radial_component)]
    
})

# Combine with the original DataFrame
df_combined = pd.concat([df.iloc[:-1].reset_index(drop=True), df_mid], axis=1)

# Save the combined DataFrame to CSV
df_combined.to_csv('combined_data_with_amplitude.csv', index=False)

# Function to split data into continuous segments
def split_into_segments(df, threshold=0.1):  # Threshold in kilometers
    segments = []
    segment = [df.iloc[0]]
    for i in range(1, len(df)):
        x1, y1 = df.iloc[i - 1][['x', 'y']]
        x2, y2 = df.iloc[i][['x', 'y']]
        distance = np.hypot(x2 - x1, y2 - y1)
        if distance > threshold:
            segments.append(pd.DataFrame(segment))
            segment = []
        segment.append(df.iloc[i])
    segments.append(pd.DataFrame(segment))
    return segments
tan = df_mid['Tangential Component']
radial = df_mid['Radial Component']


In [ ]:
# Load the combined data (if not already in memory)
df_combined = pd.read_csv('combined_data_with_amplitude.csv')

# Ensure Lithology is treated as a categorical variable
df_combined['Lithology'] = df_combined['Lithology'].astype('category')


In [ ]:

# Filtrar el dataframe
df_filtered = df_combined
#df_filtered = df_combined[df_combined['AmplitudeS'] <= amplitude_threshold]


In [ ]:
max(df_mid['Slope'])

In [ ]:
df_filtered['Amplitude-S'] = df_filtered['Amplitude-S'] * correction_factor

# Color dictionary with real lithology names
lithology_colors_real = {
    'Superficial Deposits': '#FFD700',  # Gold
    'Andesite': '#DC143C',              # Crimson
    'Volcanic Breccia': '#8B4513',      # Saddle Brown
    'Granodiorite': '#FF8C00',          # Dark Orange
    'Diorite': '#DC143C',               # Crimson
    'Diabase, Basalt': '#9370DB'        # Medium Purple
}

# Dictionary to replace lithology codes with real names
lithology_names = {
    'Qs': 'Superficial Deposits',
    'Qra': 'Andesite',
    'To': 'Volcanic Breccia',
    'Tg': 'Granodiorite',
    'Tdi': 'Diorite',
    'Td': 'Diabase, Basalt'
}

# Replace lithology names in the DataFrame
df_filtered['Lithology'] = df_filtered['Lithology'].replace(lithology_names)

# Create the boxplot figure
plt.figure(figsize=(18, 10))

# Create boxplot with custom colors
sns.boxplot(
    x='Lithology', 
    y='Amplitude-S',
    data=df_filtered, 
    palette=lithology_colors_real,
    showfliers=True
)

# Add stripplot with black points overlaid
sns.stripplot(
    x='Lithology', 
    y='Amplitude-S', 
    data=df_filtered, 
    color='black', 
    alpha=0.3, 
    jitter=True
)

# plt.xlabel('Lithology', fontsize=30)
plt.ylabel('log(Strain rate(/s))', fontsize=30)
plt.xticks(fontsize=25, rotation=45)
plt.yticks(fontsize=25)
plt.yscale('log')
# plt.title('Amplitude Distribution by Lithology', fontsize=16)

plt.tight_layout()

# Save the figure (optional)
plt.savefig('boxplot_lithology_realnames.png', dpi=300, bbox_inches='tight', format='png')

# Show the figure
plt.show()


In [ ]:
from matplotlib.colorbar import ColorbarBase
from matplotlib.colors import Normalize

# Add 'AmplitudeN' to df_mid by copying it from df (assuming same order)
# df_mid['Noise'] = df['Noise'][:len(df_mid)]

# Calculate SNR for P and S waves (using AmplitudeN as noise)
df_mid['SNR_P'] = df_mid['Amplitude-P'] / df_mid['Noise']
df_mid['SNR_S'] = df_mid['Amplitude-S'] / df_mid['Noise']

# Convert to logarithmic scale (a small constant is added to avoid log(0))
epsilon = 1e-12
df_mid['log_SNR_P'] = np.log10(df_mid['SNR_P'] + epsilon)
df_mid['log_SNR_S'] = np.log10(df_mid['SNR_S'] + epsilon)

# =====================================================
# 2) DEFINE OFFSET AND LAYER ORDER TO DISPLAY
# =====================================================
# "Slope" was removed, so the layer order is:
layer_order = [
    'Lithology',
    'Radial',
    'Tangential',
    'Slope',
    'log_SNR_P',
    'log_SNR_S'
]
base_offset = 0
spacing = 2.0
offsets = {layer: base_offset + i * spacing for i, layer in enumerate(layer_order)}

label_names = {
    'Lithology': 'Lithology (Actual cable position)',
    'Radial': 'Radial Orientation',
    'Tangential': 'Tangential Orientation',
    'Slope': 'Slope',
    'log_SNR_P': 'SNR (P-wave, log10)',
    'log_SNR_S': 'SNR (S-wave, log10)'
}
label_shift = 0.5

# =====================================================
# 3) CREATE FIGURE AND ADJUST MARGINS
# =====================================================
fig, ax = plt.subplots(figsize=(17, 14))
plt.subplots_adjust(left=0.07, right=0.93, bottom=0.07, top=0.93)

# =====================================================
# 4) PLOT LITHOLOGY MAP (using DataFrame df)
# =====================================================
# Assume df contains the columns 'Lithology', 'x', and 'y'
for lithology, color in lithology_colors.items():
    subset = df[df['Lithology'] == lithology].reset_index(drop=True)
    if len(subset) > 1:
        segments = split_into_segments(subset, threshold=0.1)
        for segment in segments:
            if len(segment) > 1:
                ax.scatter(
                    segment['x'],
                    segment['y'] + offsets['Lithology'],
                    color=color, s=50, alpha=0.8, linewidth=0.1
                )

# =====================================================
# 5) PLOT CONTINUOUS VARIABLES FROM df_mid
# =====================================================
# Slope section was removed
# a) Radial Component (cmap 'Spectral_r')
im_radial = ax.scatter(
    df_mid['x'], df_mid['y'] + offsets['Radial'],
    c=df_mid['Radial Component'], cmap='Spectral_r',
    s=50, alpha=0.5, edgecolor='k', linewidth=0.05
)

# b) Tangential Component (cmap 'Spectral_r')
im_tangential = ax.scatter(
    df_mid['x'], df_mid['y'] + offsets['Tangential'],
    c=df_mid['Tangential Component'], cmap='Spectral_r',
    s=50, alpha=0.5, edgecolor='k', linewidth=0.05
)

# c) SNR for P-wave in log scale (cmap 'Spectral_r')
im_snr_p = ax.scatter(
    df_mid['x'], df_mid['y'] + offsets['log_SNR_P'],
    c=df_mid['log_SNR_P'], cmap='Spectral_r',
    s=50, alpha=0.5, edgecolor='k', linewidth=0.05
)

# d) SNR for S-wave in log scale (cmap 'Spectral_r')
im_snr_s = ax.scatter(
    df_mid['x'], df_mid['y'] + offsets['log_SNR_S'],
    c=df_mid['log_SNR_S'], cmap='Spectral_r',
    s=50, alpha=0.5, edgecolor='k', linewidth=0.05
)

# — delete or comment this line because slope_crop is not defined —
# im_slope = ax.imshow(…)

# — instead, plot "Slope" as scatter on df_mid —
# Compute vmax at the 95th percentile to remove extreme outliers
vmax95 = np.percentile(df_mid['Slope'], 99.5)

# Scatter for slope with perceptual colormap and adjusted range
norm_slope = Normalize(vmin=0, vmax=vmax95, clip=True)
im_slope = ax.scatter(
    df_mid['x'],
    df_mid['y'] + offsets['Slope'],
    c=df_filtered['Slope'],
    cmap='Spectral_r',  # sequential perceptual
    norm=norm_slope,
    s=50,  # slightly larger markers
    edgecolor='k',
    linewidth=0.05,
    alpha=0.5
)

# Horizontal colorbar
cax_slope = ax.inset_axes([0.45, 0.05, 0.25, 0.03])
cb_slope = plt.colorbar(im_slope, cax=cax_slope, orientation='horizontal')
cb_slope.set_label('Slope (°)', fontsize=12)
cb_slope.ax.tick_params(labelsize=10)

# =====================================================
# 6) CONFIGURE AXES: REMOVE TICKS
# =====================================================
ax.set_xticks([])
ax.set_yticks([])

# =====================================================
# 7) ADD SCALE BAR AND NORTH ARROW IN LOWER LEFT CORNER
# =====================================================
# Get current axis limits
xlims = ax.get_xlim()
ylims = ax.get_ylim()

# Use plot minimums (can be slightly adjusted if desired)
x_start = xlims[0] - 1.5
y_start = ylims[0] - 1.5

# Add a scale bar (e.g., 5 km)
scalebar_length = 5  # in km
ax.plot([x_start, x_start + scalebar_length],
        [y_start, y_start],
        color='k', lw=3)
ax.text(x_start + scalebar_length/2, y_start - 0.03*(ylims[1]-ylims[0]),
        f'{scalebar_length} km', ha='center', va='top', fontsize=12)

# Add north arrow (e.g., 1 km long) in the same corner
arrow_length = 1  # in km
ax.annotate('', xy=(x_start, y_start + arrow_length), xytext=(x_start, y_start),
            arrowprops=dict(facecolor='k', width=5, headwidth=15))
ax.text(x_start, y_start + arrow_length + 0.03*(ylims[1]-ylims[0]),
        'N', ha='center', va='bottom', fontsize=12)

# =====================================================
# 8) PLOT EARTHQUAKE (keeping original location)
# =====================================================
ax.scatter(quake_x, quake_y, color='red', s=150, marker='*',
           edgecolor='black', linewidth=1.5)
ax.text(quake_x - 0.5, quake_y + 0.5, 
        'Ml 2.9, August 23, 2023', fontsize=14, color='red',
        fontweight='bold', ha='left', va='bottom',
        bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))

# =====================================================
# 9) SECTION LABELS
# =====================================================
for key, name in label_names.items():
    ax.text(
        np.min(df_mid['x']) - 0.8,
        np.min(df_mid['y']) + offsets[key] + label_shift + 1.2,
        name, fontsize=13, ha='left', fontweight='bold',
        bbox=dict(facecolor='white', alpha=0.6, edgecolor='none')
    )

# =====================================================
# 10) ADD COLORBAR FOR LITHOLOGY WITH NAMES (upper-left corner)
# =====================================================
litho_keys = list(lithology_colors.keys())
litho_keys.sort()  # Keep consistent order
colors_litho = [lithology_colors[k] for k in litho_keys]
cmap_litho = mcolors.ListedColormap(colors_litho)
norm_litho = mcolors.BoundaryNorm(np.arange(len(litho_keys)+1)-0.5, len(litho_keys))

# Horizontal colorbar with tick labels instead of numbers
cax_litho = ax.inset_axes([0.72, 0.3, 0.25, 0.03])  # coordinates in ax fraction
cb_litho = ColorbarBase(cax_litho, cmap=cmap_litho, norm=norm_litho, orientation='horizontal')
cb_litho.set_ticks(np.arange(len(litho_keys)))
cb_litho.ax.set_xticklabels(litho_keys, fontsize=12)
cb_litho.set_label('Lithology', fontsize=15)

# Add text labels for 'Paradise' and 'Nisqually Entrance'
x_paradise = df['x'].max()
y_paradise = df['y'].max() + offsets['Lithology']
x_nisqually = df['x'].min()
y_nisqually = df['y'].min() + offsets['Lithology']
ax.text(x_paradise - 0.5, y_paradise + 0.2, 'Paradise', fontsize=14, ha='left', color='black', fontweight='bold',
        bbox=dict(facecolor='white', alpha=0.8, edgecolor='none'))
ax.text(x_nisqually-0.5, y_nisqually+0.2, 'Nisqually Entrance', fontsize=14, ha='left', color='black', fontweight='bold',
        bbox=dict(facecolor='white', alpha=0.8, edgecolor='none'))

# =====================================================
# 11) REMAINING COLORBARS
# =====================================================
# Colorbar for SNR (P-wave)
cax_snr_p = ax.inset_axes([0.72, 0.18, 0.25, 0.03])
cb_snr_p = plt.colorbar(im_snr_p, cax=cax_snr_p, orientation='horizontal')
cb_snr_p.set_label('SNR (P-wave, log10)', fontsize=15)
cb_snr_p.ax.tick_params(labelsize=10)

# Colorbar for Radial component
cax_radial = ax.inset_axes([0.08, 0.94, 0.25, 0.03])
cb_radial = plt.colorbar(im_radial, cax=cax_radial, orientation='horizontal')
cb_radial.set_label('Radial Orientation', fontsize=15)
cb_radial.ax.tick_params(labelsize=10)

# Colorbar for Tangential component
cax_tangent = ax.inset_axes([0.4, 0.94, 0.25, 0.03])
cb_tangent = plt.colorbar(im_tangential, cax=cax_tangent, orientation='horizontal')
cb_tangent.set_label('Tangential Orientation', fontsize=15)
cb_tangent.ax.tick_params(labelsize=10)

# Colorbar for SNR (S-wave)
cax_snr_s = ax.inset_axes([0.72, 0.05, 0.25, 0.03])
cb_snr_s = plt.colorbar(im_snr_s, cax=cax_snr_s, orientation='horizontal')
cb_snr_s.set_label('SNR (S-wave, log10)', fontsize=15)
cb_snr_s.ax.tick_params(labelsize=10)

# =====================================================
# 12) SAVE AND SHOW FIGURE
# =====================================================
plt.savefig('cables-response-slope-corrected.png', dpi=300, bbox_inches='tight')
plt.show()
